In [1]:
import tensorflow as tf
import os

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
# drive.mount('/content/drive',force_remount=True)

In [ ]:
# data_dir = '/content/drive/My Drive/Data'


In [ ]:
# os.listdir(data_dir)

['HAM10000_images_part_2', 'HAM10000_images_part_1', 'HAM10000_images', 'HAM']

In [7]:
# import cv2
import imghdr

In [62]:
img_exts=['jpg','jpeg','png','bmp']

In [ ]:
# for image_class in os.listdir(data_dir):
#   for image in os.listdir(os.path.join(data_dir, image_class)):
#     image_path = os.path.join(data_dir,image_class, image)
#     try:
#       img = cv2.imread(image_path)
#       tip = imghdr.what(image_path)
#       if tip not in img_exts:
#         print('Image not in ext list {}'.format(image_path))
#         os.remove(image_path)
#     except Exception as e:
#       print('Issue with image {}'.format(image_path))
#       os.remove(image_path)

In [ ]:
# tf.data.Dataset??

In [3]:
import numpy as np
from matplotlib  import pyplot as plt

In [24]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras import optimizers
from keras.applications import MobileNetV2

In [26]:
base_model = MobileNetV2(include_top=False, weights="imagenet", input_shape=(224,224,3))
model = Sequential()

In [27]:
# Conv2D(no_of_filters, size_of_filters, stride)

model.add(base_model)
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.40))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.40))
model.add(Dense(7, activation='sigmoid'))


In [22]:
# # Conv2D(no_of_filters, size_of_filters, stride)

# model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
# model.add(MaxPooling2D())

# model.add(Conv2D(32, (3,3), 1, activation='relu'))
# model.add(MaxPooling2D())

# model.add(Conv2D(16, (3,3), 1, activation='relu'))
# model.add(MaxPooling2D())

# model.add(Flatten())

# model.add(Dense(256, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

In [28]:
import pandas as pd

In [29]:
label_path = './HAM10000_metadata.csv'
label_data = pd.read_csv(label_path)
label_data['image_full_name'] = label_data['image_id'] + '.jpg'
X = label_data[['image_full_name','dx','lesion_id']]

In [30]:
from sklearn.model_selection import train_test_split
Y = X.pop('dx').to_frame()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [31]:
train = pd.concat([X_train, y_train],axis=1)
# val = pd.concat([X_val, y_val],axis=1)
test = pd.concat([X_test, y_test],axis=1)

In [31]:
# from sklearn.preprocessing import LabelEncoder
# encoder = LabelEncoder()
# encoder.fit(val['dx'])
# name_as_indexes_test = encoder.transform(val['dx'])
# val['label'] = name_as_indexes_test

In [32]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(test['dx'])
name_as_indexes_test = encoder.transform(test['dx'])
test['label'] = name_as_indexes_test

In [33]:
from keras.preprocessing.image import ImageDataGenerator
train_generator = ImageDataGenerator()

In [52]:
# data = tf.keras.utils.image_dataset_from_directory('./HAM10000_images')

In [34]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [14]:
# import sys
# from PIL import Image
# # sys.modules['Image'] = Image 

In [36]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 conv2d_6 (Conv2D)           (None, 5, 5, 64)          737344    
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 2, 2, 64)         0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 2, 2, 64)          0         
                                                                 
 flatten_2 (Flatten)         (None, 256)               0         
                                                                 
 dense_4 (Dense)             (None, 128)              

In [37]:

img_size=(224,224)
gen=ImageDataGenerator(rescale=1/255,rotation_range= 10, zoom_range=0.1, width_shift_range=0.0, height_shift_range=0.0)

train_gen=gen.flow_from_dataframe(train, directory='./HAM10000_images/', x_col='image_full_name', y_col='dx', shuffle=True,
                                 target_size=img_size, class_mode='categorical', batch_size=32, subset='training')
# valid_gen=gen.flow_from_dataframe(val, directory='./HAM10000_images/', x_col='image_full_name', y_col='dx', shuffle=True,
#                                  target_size=img_size, class_mode='categorical', batch_size=32, subset='validation')
test_gen=gen.flow_from_dataframe(test, directory='./HAM10000_images/', x_col='image_full_name', y_col='dx', shuffle=False,
                                 target_size=img_size, class_mode='categorical', batch_size=32)

Found 8012 validated image filenames belonging to 7 classes.
Found 2003 validated image filenames belonging to 7 classes.


In [38]:
history=model.fit(train_gen, verbose=1, epochs=10)

Epoch 1/10
251/251 [==============================] - 133s 504ms/step - loss: 0.2493 - accuracy: 0.6781
Epoch 2/10
251/251 [==============================] - 110s 438ms/step - loss: 0.1826 - accuracy: 0.7421
Epoch 3/10
251/251 [==============================] - 107s 425ms/step - loss: 0.1631 - accuracy: 0.7692
Epoch 4/10
251/251 [==============================] - 114s 455ms/step - loss: 0.1712 - accuracy: 0.7650
Epoch 5/10
251/251 [==============================] - 102s 406ms/step - loss: 0.1520 - accuracy: 0.7822
Epoch 6/10
251/251 [==============================] - 113s 450ms/step - loss: 0.1430 - accuracy: 0.7981
Epoch 7/10
251/251 [==============================] - 101s 400ms/step - loss: 0.1388 - accuracy: 0.8103
Epoch 8/10
251/251 [==============================] - 102s 406ms/step - loss: 0.1579 - accuracy: 0.7812
Epoch 9/10
251/251 [==============================] - 105s 419ms/step - loss: 0.1403 - accuracy: 0.8037
Epoch 10/10
251/251 [==============================] - 103s 410m

In [39]:
acc=model.evaluate(test_gen)[1]* 100
print (acc)

63/63 [==============================] - 33s 505ms/step - loss: 0.4000 - accuracy: 0.6680
66.79980158805847


In [40]:
model.save("model_skin.h5")